<a href="https://colab.research.google.com/github/NickMattiss/ai-ml-principles-exercises/blob/main/log_ml_train_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [3]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import wandb

In [ ]:
# wandb.init(project="log_ml_train_with_wandb")
wandb.login()

In [ ]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import tensorflow
tensorflow.__version__

'2.9.2'

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [ ]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [ ]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [ ]:
batch_size = 128
epochs = 5

In [ ]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

In [ ]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])

Epoch 1/5
422/422 [==============================] - 47s 109ms/step - loss: 0.2915 - accuracy: 0.9167 - val_loss: 0.0790 - val_accuracy: 0.9797
Epoch 2/5
422/422 [==============================] - 47s 111ms/step - loss: 0.0797 - accuracy: 0.9760 - val_loss: 0.0600 - val_accuracy: 0.9832
Epoch 3/5
422/422 [==============================] - 46s 108ms/step - loss: 0.0576 - accuracy: 0.9832 - val_loss: 0.0562 - val_accuracy: 0.9847
Epoch 4/5
422/422 [==============================] - 45s 108ms/step - loss: 0.0488 - accuracy: 0.9852 - val_loss: 0.0502 - val_accuracy: 0.9855
Epoch 5/5
422/422 [==============================] - 46s 108ms/step - loss: 0.0403 - accuracy: 0.9876 - val_loss: 0.0414 - val_accuracy: 0.9875
